# Analyzing Well Bundles

## 1. Importing / Installing Packages

In [21]:
import pandas as pd # Importing pandas package

# Set the maximum number of columns to display to None
pd.set_option('display.max_columns', None)

import numpy as np # Importing numpy package

from typing import Dict, Tuple, List, Union # Importing specific types from typing module

import re # Importing regular expression package

from src.database_manager import DatabricksOdbcConnector # Importing DatabricksOdbcConnector class from database_manager module
from src.utils import reorder_columns # Importing reorder_columns function from utils module

from scipy.spatial.distance import cdist # Importing cdist function from scipy package

import time

import pyproj # Importing pyproj package

## 2. Loading Excel/csv into Pandas DataFrame

In [22]:
# List of column names for the DataFrame
header_colms = ['Well Name', 'Chosen ID', 'Lease Name', 'RSV_CAT', 'Bench', 'First Prod Date', 'Hole Direction']

In [23]:
df_raw = pd.read_excel('MB Header.xlsx',dtype={'Chosen ID':str},parse_dates=['First Prod Date'], usecols=header_colms) # Reading an Excel file into a pandas DataFrame

In [24]:
df_raw.rename(columns={
    'Chosen ID':'ChosenID',
    'Well Name':'WellName',
    'RSV_CAT':'RES_CAT',
    'Bench':'Landing_Zone',
    'First Prod Date':'FirstProdDate',
    'Hole Direction':'HoleDirection',
    'Lease Name':'LeaseName'
}, inplace=True) # Renaming columna in the DataFrame

In [25]:
df_raw['First Prod Date'] = pd.to_datetime(df_raw['FirstProdDate']) # Converting 'FirstProdDate' column to datetime format

In [26]:
df_raw.shape

(7703, 8)

## 3. Data Preprocessing

### 3.1. Creating DSU Columns

In [27]:
# Creating DSU columns names from Lease Name columns

df_raw['DSU'] = df_raw['LeaseName'].apply(
    lambda x: re.sub(r'[^a-zA-Z\s]', ' ',  # Remove special characters, keep letters and spaces
                     re.match(r'([^\d]+)', str(x)).group(1) if pd.notna(x) and re.match(r'([^\d]+)', str(x)) else str(x))  
                    .strip()  # Strip leading/trailing spaces
).replace(r'\s+', ' ', regex=True)  # Collapse multiple spaces into a single space

# Placing DSU next to LeaseName
df_raw = reorder_columns(df=df_raw, columns_to_move=['DSU'], reference_column='LeaseName')

## 4. Feature Engineering

### 4.1. Defining Functions that is used in calculation for i-k pair dataframe

In [28]:
def extract_heel_toe_mid_lat_lon(well_trajectory: pd.DataFrame) -> pd.DataFrame:
    """
    Extract the heel, toe, and mid-point latitude/longitude for each ChosenID in the well trajectory DataFrame.

    Parameters:
    well_trajectory: pd.DataFrame
        DataFrame containing well trajectory data, including 'ChosenID', 'md', 'latitude', and 'longitude'.

    Returns:
    pd.DataFrame
        A DataFrame with 'ChosenID', 'Heel_Lat', 'Heel_Lon', 'Toe_Lat', 'Toe_Lon', 'Mid_Lat', 'Mid_Lon'.

    Example:
    >>> data = {
    ...     "ChosenID": [1001, 1001, 1001, 1002, 1002],
    ...     "md": [5000, 5100, 5200, 6000, 6100],
    ...     "latitude": [31.388, 31.389, 31.387, 31.400, 31.401],
    ...     "longitude": [-103.314, -103.315, -103.316, -103.318, -103.319]
    ... }
    >>> df = pd.DataFrame(data)
    >>> extract_heel_toe_mid_lat_lon(df)
       ChosenID  Heel_Lat  Heel_Lon  Toe_Lat  Toe_Lon  Mid_Lat  Mid_Lon
    0     1001    31.388  -103.314   31.387  -103.316  31.3875 -103.315
    1     1002    31.400  -103.318   31.401  -103.319  31.4005 -103.3185
    """
    # Ensure the data is sorted by MD in ascending order
    well_trajectory = well_trajectory.sort_values(by=["ChosenID", "md"], ascending=True)

    # Group by 'ChosenID' and extract heel/toe lat/lon
    heel_toe_df = (
        well_trajectory.groupby("ChosenID")
        .agg(
            heel_lat=("latitude", "first"),
            heel_lon=("longitude", "first"),
            toe_lat=("latitude", "last"),
            toe_lon=("longitude", "last"),
        )
        .reset_index()
    )

    # Calculate midpoints
    heel_toe_df["mid_Lat"] = (heel_toe_df["heel_lat"] + heel_toe_df["toe_lat"]) / 2
    heel_toe_df["mid_Lon"] = (heel_toe_df["heel_lon"] + heel_toe_df["toe_lon"]) / 2

    return heel_toe_df

In [29]:
def get_direction(lat1: np.ndarray, lon1: np.ndarray, lat2: np.ndarray, lon2: np.ndarray) -> np.ndarray:
    """
    Determine the relative direction of (lat2, lon2) with respect to (lat1, lon1).
    
    Parameters:
    lat1, lon1: np.ndarray
        Latitude and longitude of the first well.
    lat2, lon2: np.ndarray
        Latitude and longitude of the second well.
    
    Returns:
    np.ndarray
        Array indicating the direction (e.g., North, South, East, West) of well B relative to well A.
    """
    lat_diff = lat1 - lat2
    lon_diff = lon1 - lon2

    conditions = [
        np.abs(lat_diff) > np.abs(lon_diff), # Determines if movement is more North/South
        lat_diff > 0, # B is South of A
        lon_diff > 0  # B is West of A
    ]

    choices = ["N", "S", "E", "W"]
    
    return np.select(
        [conditions[0] & conditions[1], # More movement in North/South direction & B is South of A
         conditions[0] & ~conditions[1], # More movement in North/South direction & B is North of A
         ~conditions[0] & conditions[2], # More movement in East/West direction & B is West of A
         ~conditions[0] & ~conditions[2]], # More movement in East/West direction & B is East of A
        choices
    )

In [30]:
def calculate_drill_direction_vectorized(well_trajectories: Dict[str, pd.DataFrame], i_indices: np.ndarray) -> np.ndarray:
    """
    Optimized vectorized function to determine the drilling direction of multiple wells using NumPy operations.
    
    Parameters:
    well_trajectories: Dict[str, pd.DataFrame]
        Dictionary containing well trajectory data indexed by ChosenID.
    i_indices: np.ndarray
        Array of ChosenID whose drill directions need to be calculated.
    
    Returns:
    np.ndarray
        Array containing "EW" (East-West) or "NS" (North-South) for each well.
    """
    start_time = time.time()

    # 🚀 Precompute medians for all wells at once
    all_data = pd.concat(well_trajectories.values(), keys=well_trajectories.keys()).reset_index(level=0)
    azimuth_medians = all_data.groupby("level_0")["azimuth"].median().to_dict()

    step1_time = time.time()
    print(f"✅ Step 1: Precomputed azimuth medians in {step1_time - start_time:.4f} seconds.")

    # 🚀 Fast lookup using NumPy
    azimuth_values = np.array([azimuth_medians.get(i, np.nan) for i in i_indices])

    step2_time = time.time()
    print(f"✅ Step 2: Retrieved azimuth values in {step2_time - step1_time:.4f} seconds.")

    # 🚀 Apply vectorized conditions
    conditions = (45 <= azimuth_values) & (azimuth_values < 135) | (225 <= azimuth_values) & (azimuth_values < 315)
    drill_directions = np.where(np.isnan(azimuth_values), "Unknown", np.where(conditions, "EW", "NS"))

    step3_time = time.time()
    print(f"✅ Step 3: Assigned drill directions in {step3_time - step2_time:.4f} seconds.")
    
    total_time = time.time() - start_time
    print(f"🚀 Total Execution Time: {total_time:.4f} seconds.")

    return drill_directions

In [31]:
def optimized_calculate_3D_distance_matrix(
    trajectories: Dict[str, pd.DataFrame], i_indices: np.ndarray, k_indices: np.ndarray
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Fully vectorized 3D distance calculations for well pairs using NumPy and Pandas.
    
    Parameters:
    trajectories: Dict[str, pd.DataFrame]
        Dictionary containing well trajectory data indexed by well ID.
    i_indices: np.ndarray
        Array of well IDs representing the first well in each pair.
    k_indices: np.ndarray
        Array of well IDs representing the second well in each pair.
    
    Returns:
    Tuple[np.ndarray, np.ndarray, np.ndarray]
        - Horizontal distances between the well pairs.
        - Vertical distances between the well pairs.
        - 3D distances between the well pairs.
    """
    # 🚀 Precompute mean (midpoint) for each well ID across all wells at once
    all_trajectories_df = pd.concat(trajectories.values(), keys=trajectories.keys()).reset_index(drop=True)

    midpoints_df = all_trajectories_df.groupby("ChosenID")[["x", "y", "tvd"]].mean()

    # Convert to NumPy arrays for fast lookup
    well_ids = midpoints_df.index.to_numpy()
    midpoints = midpoints_df.to_numpy()

    # Create a mapping from well ID to its index
    well_id_to_idx = {well_id: idx for idx, well_id in enumerate(well_ids)}

    # Efficiently extract midpoints using NumPy indexing
    mid_A = midpoints[np.array([well_id_to_idx[i] for i in i_indices])]
    mid_B = midpoints[np.array([well_id_to_idx[k] for k in k_indices])]

    # Compute distances
    vertical_distances = np.abs(mid_A[:, 2] - mid_B[:, 2])
    mid_B[:, 2] = mid_A[:, 2]  # Align Well B to Well A’s TVD

    horizontal_distances = np.linalg.norm(mid_A[:, :2] - mid_B[:, :2], axis=1)
    total_3D_distances = np.sqrt(horizontal_distances**2 + vertical_distances**2)

    return horizontal_distances, vertical_distances, total_3D_distances

In [32]:
def create_i_k_pairs(df: pd.DataFrame, trajectories: Union[Dict[str, pd.DataFrame], pd.DataFrame]) -> pd.DataFrame:
    """
    Generate the i_k_pairs DataFrame, computing horizontal and vertical distances, 
    3D distances, drilling directions, and relative directions between well pairs.
    
    Parameters:
    df: pd.DataFrame
        DataFrame containing well metadata with:
        - "ChosenID" (str): Unique well identifier.

    trajectories: Union[Dict[str, pd.DataFrame], pd.DataFrame]
        Either:
        - A dictionary mapping well IDs ("ChosenID") to trajectory DataFrames.
        - A single DataFrame containing all trajectory data (must have "ChosenID" column).
        
    Each trajectory DataFrame should include:
    - "md" (float): Measured depth.
    - "tvd" (float): True vertical depth.
    - "inclination" (float): Inclination angle in degrees.
    - "azimuth" (float): represents the drilling direction.
    - "latitude" (float): Latitude values, define the geographical position.
    - "longitude" (float): Longitude values, define the geographical position.
    - "x" (float): X-coordinate in a Cartesian coordinate system.
    - "y" (float): Y-coordinate in a Cartesian coordinate system.
    
    Returns:
    pd.DataFrame
        DataFrame containing pairs of wells (`i_uwi`, `k_uwi`) with their computed distances 
        and directional relationships.
    """
    start_time = time.time()
    
    # Convert to dictionary if input is a DataFrame
    step1_start = time.time()
    if isinstance(trajectories, pd.DataFrame):
        if "ChosenID" not in trajectories.columns:
            raise ValueError("🚨 Error: Trajectory DataFrame must contain a 'ChosenID' column.")
        trajectories = {cid: group for cid, group in trajectories.groupby("ChosenID")}
    step1_end = time.time()
    print(f"✅ Step 1: Converted trajectory DataFrame to dictionary in {step1_end - step1_start:.4f} seconds.")

    # Get unique ChosenIDs from df
    step2_start = time.time()
    chosen_ids = df["ChosenID"].unique()
    missing_ids = [cid for cid in chosen_ids if cid not in trajectories]

    if missing_ids:
        print(f"⚠️ The following ChosenIDs do not exist in the trajectory data and will be excluded: {missing_ids}")

    df = df[df["ChosenID"].isin(trajectories)] # Filter out missing IDs in the DataFrame
    chosen_ids = df["ChosenID"].unique() # Update chosen_ids without missing IDs
    step2_end = time.time()
    print(f"✅ Step 2: Extracted unique ChosenIDs in {step2_end - step2_start:.4f} seconds.")

    # Generate all possible pairs (excluding self-comparison)
    step3_start = time.time()
    i_uwi, k_uwi = np.meshgrid(chosen_ids, chosen_ids, indexing='ij')
    i_uwi, k_uwi = i_uwi.ravel(), k_uwi.ravel()

    # Remove self-comparisons
    valid_mask = i_uwi != k_uwi
    i_uwi, k_uwi = i_uwi[valid_mask], k_uwi[valid_mask]
    step3_end = time.time()
    print(f"✅ Step 3: Generated well pairs in {step3_end - step3_start:.4f} seconds.")

    # 🚀 Optimized Heel/Toe Extraction (Vectorized)
    step4_start = time.time()
    heel_toe_df = pd.concat(
        [extract_heel_toe_mid_lat_lon(trajectories[cid]) for cid in chosen_ids], ignore_index=True
    )
    heel_toe_dict = heel_toe_df.set_index("ChosenID").to_dict(orient="index")
    step4_end = time.time()
    print(f"✅ Step 4: Heel/Toe extraction took {step4_end - step4_start:.4f} seconds.")

    # Efficiently extract values using vectorized lookups
    step5_start = time.time()
    # heel_lat_i = np.array([heel_toe_dict[i]["heel_lat"] for i in i_uwi])
    # heel_lon_i = np.array([heel_toe_dict[i]["heel_lon"] for i in i_uwi])
    # toe_lat_k = np.array([heel_toe_dict[k]["toe_lat"] for k in k_uwi])
    # toe_lon_k = np.array([heel_toe_dict[k]["toe_lon"] for k in k_uwi])
    mid_lat_i = np.array([heel_toe_dict[i]["mid_Lat"] for i in i_uwi])
    mid_lon_i = np.array([heel_toe_dict[i]["mid_Lon"] for i in i_uwi])
    mid_lat_k = np.array([heel_toe_dict[k]["mid_Lat"] for k in k_uwi])
    mid_lon_k = np.array([heel_toe_dict[k]["mid_Lon"] for k in k_uwi])
    step5_end = time.time()
    print(f"✅ Step 5: Heel/Toe dictionary lookup took {step5_end - step5_start:.4f} seconds.")

    # 🚀 Optimized Distance Calculation (Fully Vectorized)
    step6_start = time.time()
    horizontal_dist, vertical_dist, total_3D_dist = optimized_calculate_3D_distance_matrix(trajectories, i_uwi, k_uwi)
    step6_end = time.time()
    print(f"✅ Step 6: Distance calculations took {step6_end - step6_start:.4f} seconds.")

    # Compute drill directions
    step7_start = time.time()
    drill_directions = calculate_drill_direction_vectorized(trajectories, i_uwi)
    step7_end = time.time()
    print(f"✅ Step 7: Drill direction calculation took {step7_end - step7_start:.4f} seconds.")

    # Determine directional relationship
    step8_start = time.time()
    ward_of_i = get_direction(mid_lat_i, mid_lon_i, mid_lat_k, mid_lon_k)
    step8_end = time.time()
    print(f"✅ Step 8: Directional relationship calculation took {step8_end - step8_start:.4f} seconds.")

    # Create DataFrame
    step9_start = time.time()
    result_df = pd.DataFrame({
        "i_uwi": i_uwi,
        "k_uwi": k_uwi,
        "horizontal_dist": horizontal_dist,
        "vertical_dist": vertical_dist,
        "3D_ft_to_same": total_3D_dist,
        "drill_direction": drill_directions,
        "ward_of_i": ward_of_i
    })
    step9_end = time.time()
    print(f"✅ Step 9: Created result DataFrame in {step9_end - step9_start:.4f} seconds.")

    total_time = time.time() - start_time
    print(f"🚀 Total Execution Time: {total_time:.4f} seconds.")

    return result_df

In [33]:
def calculate_overlap(well_A: pd.DataFrame, well_B: pd.DataFrame) -> float:
    """
    Calculate the percentage overlap between two horizontal wellbores.
    
    Parameters:
    well_A: pd.DataFrame
        Well trajectory data for Well A, including 'MD' (Measured Depth) and 'Inclination'.
    well_B: pd.DataFrame
        Well trajectory data for Well B, including 'MD' (Measured Depth) and 'Inclination'.
    
    Returns:
    float:
        Percentage of overlap relative to the shorter lateral.
    """
    if well_A.empty or well_B.empty:
        return 0.0

    start_A, end_A = well_A["MD"].min(), well_A["MD"].max()
    start_B, end_B = well_B["MD"].min(), well_B["MD"].max()

    overlap_start = max(start_A, start_B)
    overlap_end = min(end_A, end_B)

    if overlap_start >= overlap_end:
        return 0.0

    overlap_length = overlap_end - overlap_start
    shorter_length = min(end_A - start_A, end_B - start_B)

    return (overlap_length / shorter_length) * 100 if shorter_length > 0 else 0.0

### 4.2. Defining Functions that is used to compute Lat/Lon to UTM Co-Ordinates

In [34]:
def determine_utm_zone(longitude: float) -> int:
    """
    Determines the UTM zone based on a given longitude.
    """
    return int((longitude + 180) / 6) + 1


def batch_latlon_to_utm(lat: np.ndarray, lon: np.ndarray, utm_zone: int) -> Tuple[np.ndarray, np.ndarray]:
    """
    Converts arrays of latitudes and longitudes to UTM coordinates in meters for a given UTM zone.
    """
    proj_utm = pyproj.Transformer.from_crs(
        "EPSG:4326", f"EPSG:326{utm_zone}", always_xy=True
    )
    
    return proj_utm.transform(lon, lat)


def compute_utm_coordinates(df: pd.DataFrame) -> pd.DataFrame:
    """
    Computes UTM (x, y, z) coordinates for multiple wells, using surface location to determine UTM zones.
    Converts UTM coordinates from meters to feet. Uses vectorized batch processing for performance.

    Parameters:
    - df (pd.DataFrame): Original directional survey DataFrame.

    Returns:
    - pd.DataFrame: DataFrame with all original columns + x, y, z (in feet), and utm_zone.
    """
    start_time = time.time()  # Start timing

    # Step 1: Sort dataframe by md to identify surface location
    df = df.sort_values(by=["ChosenID", "md"], ascending=[True, True])
    
    # Step 2: Determine UTM zones using the surface location (first row per well)
    surface_locs = df.groupby("ChosenID").first()[["latitude", "longitude"]]
    surface_locs["utm_zone"] = surface_locs["longitude"].apply(determine_utm_zone)

    # Merge UTM zones back into the original dataframe
    df = df.merge(surface_locs[["utm_zone"]], on="ChosenID", how="left")

    print(f"✅ Determined UTM zones in {time.time() - start_time:.4f} seconds.")

    # Step 3: Batch transformation for each unique UTM zone
    start_transform_time = time.time()
    unique_zones = df["utm_zone"].unique()
    utm_converters: Dict[int, Tuple[np.ndarray, np.ndarray]] = {}

    for zone in unique_zones:
        subset = df[df["utm_zone"] == zone]
        easting, northing = batch_latlon_to_utm(subset["latitude"].values, subset["longitude"].values, zone)
        utm_converters[zone] = (easting, northing)

    print(f"✅ Performed batch EPSG transformations in {time.time() - start_transform_time:.4f} seconds.")

    # Step 4: Assign the converted coordinates back to the DataFrame
    start_assign_time = time.time()
    df["x"], df["y"] = np.zeros(len(df)), np.zeros(len(df))

    for zone in unique_zones:
        mask = df["utm_zone"] == zone
        df.loc[mask, "x"], df.loc[mask, "y"] = utm_converters[zone]

    print(f"✅ Assigned transformed coordinates in {time.time() - start_assign_time:.4f} seconds.")

    # Step 5: Convert UTM coordinates from meters to feet (Conversion factor: 1 meter = 3.28084 feet)
    df["x"] *= 3.28084
    df["y"] *= 3.28084
    
    df["z"] = -df["tvd"] # Elevation is negative TVD

    print(f"✅ Total execution time: {time.time() - start_time:.4f} seconds.")

    return df


def filter_after_heel_point(df: pd.DataFrame) -> pd.DataFrame:
    """
    Filters the dataframe to include all rows for each ChosenID where the first occurrence 
    of either '80' or 'heel' appears in the point_type column and all subsequent rows.

    Parameters:
    df (pd.DataFrame): A dataframe containing directional survey data with a 'ChosenID' column and 'point_type' column.

    Returns:
    pd.DataFrame: Filtered dataframe containing rows from the first occurrence of '80' or 'heel' onward.
    """

    # Convert 'point_type' to lowercase and check for '80' or 'heel'
    mask = df['point_type'].str.lower().str.contains(r'80|heel', regex=True, na=False)

    # Identify the first occurrence for each ChosenID
    idx_start = df[mask].groupby('ChosenID', sort=False).head(1).index

    # Create a mapping of ChosenID to the starting index
    start_idx_map = dict(zip(df.loc[idx_start, 'ChosenID'], idx_start))

    # Create a boolean mask using NumPy to filter rows
    chosen_ids = df['ChosenID'].values
    indices = np.arange(len(df))

    # Get the minimum start index for each row's ChosenID
    start_indices = np.vectorize(start_idx_map.get, otypes=[float])(chosen_ids)

    # Mask rows where index is greater than or equal to the start index
    valid_rows = indices >= start_indices

    return df[valid_rows].reset_index(drop=True)

## 5. Testinig

In [35]:
df_raw['RES_CAT'].unique()

array(['01PDP', '02PDNP', '02PA', 'OLD DUC', '03PUD', '03PA', '05PA'],
      dtype=object)

In [36]:
# Filtering df_raw to include only certain RSV_CAT values
df_raw = df_raw[df_raw['RES_CAT'].isin(['01PDP', '02PDNP', '03PUD'])].reset_index(drop=True).copy()

In [37]:
# Importing Directional Survey data from Databricks

databricks = DatabricksOdbcConnector()

# Filtering only Horizontal wells and getting their apis
chosen_ids = ", ".join(f"'{id}'" for id in df_raw[df_raw['HoleDirection']=='H']['ChosenID'].unique())

try:
    databricks.connect()

    query = f"""
    SELECT
        LEFT(uwi, 10) AS ChosenID, 
        station_md_uscust AS md, 
        station_tvd_uscust AS tvd,
        inclination, 
        azimuth, 
        latitude, 
        longitude, 
        x_offset_uscust AS `deviation_E/W`,
        ew_direction,
        y_offset_uscust AS `deviation_N/S`,
        ns_direction,
        point_type
        
    FROM ihs_sp.well.well_directional_survey_station
    WHERE LEFT(uwi, 10) IN ({chosen_ids})
    order by uwi, md;
    """

    df_directional = databricks.execute_query(query)

except Exception as e:
    print(f"Error: {e}")

finally:
    databricks.close_connection()

c:\users\apoorva.saxena\onedrive - sitio royalties\desktop\project - apoorva\python\parent_child_spacing\src\database_manager.py:85: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(sql_query, self.connection)


In [38]:
df_with_utm = compute_utm_coordinates(df_directional)

✅ Determined UTM zones in 1.0048 seconds.
✅ Performed batch EPSG transformations in 0.3404 seconds.
✅ Assigned transformed coordinates in 0.0172 seconds.
✅ Total execution time: 1.3828 seconds.


In [39]:
filtered_df = filter_after_heel_point(df_with_utm)

In [40]:
# extract_heel_toe_mid_lat_lon(filtered_df[['ChosenID','md','tvd','inclination','azimuth','latitude','longitude','x','y','z']]).to_csv(r"C:\Users\Apoorva.Saxena\OneDrive - Sitio Royalties\Desktop\Project - Apoorva\MB Investigation\HeelToe.csv", index=False)

In [41]:
df_ik_pairs = create_i_k_pairs(df=df_raw, trajectories=filtered_df[['ChosenID','md','tvd','inclination','azimuth','latitude','longitude','x','y','z']])

✅ Step 1: Converted trajectory DataFrame to dictionary in 0.3125 seconds.
⚠️ The following ChosenIDs do not exist in the trajectory data and will be excluded: ['4232944094', '4232941762', '4232941872', '4246134423', '4222740685', '4222740681', '4232943669', '4222741542', '4246142022', '4246142016', '4222741585', '4231744184', '4232945684', '4238341181', '4222741952', '4246142273', '4246142272', '4231745076', '4232946465', '4231745686', '4231745690', '4231745685', '4246142636', '4246142644', '4231745868', '4231745970', '4231745971', '4232946926', '4231746124', '4232947036', '4231746232', '4231746283', '4231746288', '4232947200', '4231746448', '4246143025']
✅ Step 2: Extracted unique ChosenIDs in 0.0050 seconds.
✅ Step 3: Generated well pairs in 1.1121 seconds.
✅ Step 4: Heel/Toe extraction took 19.9128 seconds.
✅ Step 5: Heel/Toe dictionary lookup took 19.1727 seconds.
✅ Step 6: Distance calculations took 11.9329 seconds.
✅ Step 1: Precomputed azimuth medians in 0.2777 seconds.
✅ Step 2

In [42]:
df_ik_pairs[df_ik_pairs['horizontal_dist']<=21120].to_csv(r"C:\Users\Apoorva.Saxena\OneDrive - Sitio Royalties\Desktop\Project - Apoorva\MB Investigation\MB_ikPairs_v4.csv", index=False)

In [43]:
df_ik_pairs.to_csv(r"C:\Users\Apoorva.Saxena\OneDrive - Sitio Royalties\Desktop\Project - Apoorva\MB Investigation\MB_ikPairs_FULL_v1.csv", index=False)

### 5.1. Polygon Output

In [25]:
header = df_raw.copy()
ik_pair = df_ik_pairs.copy()

# Sort header for efficient merging
header_sorted = header.sort_values(by="ChosenID")

# Merge Landing_Zone information only once
df_merge = ik_pair.merge(
    header_sorted[["ChosenID", "Landing_Zone"]].rename(columns={'ChosenID': 'i_uwi'}),
    how="left", on="i_uwi"
).merge(
    header_sorted[["ChosenID", "Landing_Zone"]].rename(columns={'ChosenID': 'k_uwi'}),
    how="left", on="k_uwi", suffixes=("_i", "_k")
)

# Keep only the smallest `horizontal_dist` per (i_uwi, k_uwi)
df_grouped_horiz = df_merge.groupby(['i_uwi', 'k_uwi', 'Landing_Zone_i', 'Landing_Zone_k'])[['horizontal_dist']].min().reset_index()

# Keep only the smallest `vertical_dist` per (i_uwi, k_uwi)
df_grouped_vert = df_merge.groupby(['i_uwi', 'k_uwi', 'Landing_Zone_i', 'Landing_Zone_k'])[['vertical_dist']].min().reset_index()

# Sort DataFrame efficiently
df_sorted_horiz = df_grouped_horiz.sort_values(by=['i_uwi', 'horizontal_dist']).reset_index(drop=True)
df_sorted_vert = df_grouped_vert.sort_values(by=['i_uwi', 'vertical_dist']).reset_index(drop=True)

# Extract top 2 `k_uwi` values separately for same & different landing zones
df_same = df_sorted_horiz[df_sorted_horiz["Landing_Zone_i"] == df_sorted_horiz["Landing_Zone_k"]].groupby("i_uwi").head(2)

df_diff = df_sorted_vert[df_sorted_vert["Landing_Zone_i"] != df_sorted_vert["Landing_Zone_k"]].groupby("i_uwi").head(2)

# Pivot for both same and different Landing Zones
def pivot_top_two(df, zone_type):
    """Helper function to pivot grouped data into `k_uwi_*`, `horizontal_dist_*`, and `vertical_dist_*` columns."""
    df["rank"] = df.groupby("i_uwi").cumcount() + 1  # Create ranking (1 or 2) for each i_uwi

    # Identify available columns for pivoting
    available_cols = ["k_uwi"] + list(df.columns.intersection(["horizontal_dist", "vertical_dist"]))

    # Pivot only available columns
    df_pivot = df.pivot(index="i_uwi", columns="rank", values=available_cols)

    # Correct column renaming based on the landing zone type
    df_pivot.columns = [
        f"k_uwi_{zone_type}{col[1]}" if col[0] == "k_uwi"
        else f"horizontal_dist_{zone_type}{col[1]}" if col[0] == "horizontal_dist"
        else f"vertical_dist_{zone_type}{col[1]}"
        for col in df_pivot.columns
    ]

    return df_pivot.reset_index()


# Pivot for both same and different Landing Zones
df_same_pivot = pivot_top_two(df_same, "same")
df_diff_pivot = pivot_top_two(df_diff, "near")

final_df = df_same_pivot.merge(df_diff_pivot, how="outer", on="i_uwi")

# Merging well attributes for `i_uwi`
final_df = final_df.merge(header_sorted[['ChosenID','WellName','DSU','RES_CAT','Landing_Zone','FirstProdDate']].rename(columns={'ChosenID':'i_uwi'}), 
                          how="left")

# Merging well attributes for `k_uwi_same1`
final_df = final_df.merge(header_sorted[['ChosenID','WellName','RES_CAT','Landing_Zone','FirstProdDate']].rename(columns={'ChosenID':'k_uwi_same1'}), 
                          how="left", suffixes=("", "_same1"), right_on="k_uwi_same1", left_on="k_uwi_same1")

# Merging well attributes for , `k_uwi_same2`
final_df = final_df.merge(header_sorted[['ChosenID','WellName','RES_CAT','Landing_Zone','FirstProdDate']].rename(columns={'ChosenID':'k_uwi_same2'}), 
                          how="left", suffixes=("", "_same2"), right_on="k_uwi_same2", left_on="k_uwi_same2")

# Merging well attributes for `k_uwi_near1`
final_df = final_df.merge(header_sorted[['ChosenID','WellName','RES_CAT','Landing_Zone','FirstProdDate']].rename(columns={'ChosenID':'k_uwi_near1'}), 
                          how="left", suffixes=("", "_near1"), right_on="k_uwi_near1", left_on="k_uwi_near1")

# Merging well attributes for `k_uwi_near2`
final_df = final_df.merge(header_sorted[['ChosenID','WellName','RES_CAT','Landing_Zone','FirstProdDate']].rename(columns={'ChosenID':'k_uwi_near2'}), 
                          how="left", suffixes=("", "_near2"), right_on="k_uwi_near2", left_on="k_uwi_near2")

# Merging ik_pair
final_df = final_df.merge(ik_pair[['i_uwi','k_uwi','vertical_dist','3D_ft_to_same']].rename(columns={
    'k_uwi':'k_uwi_same1',
    'vertical_dist':'vertical_dist_same1',
    '3D_ft_to_same':'3D_ft_to_same1'
}), 
               how='left', left_on=['i_uwi','k_uwi_same1'], right_on=['i_uwi','k_uwi_same1'], suffixes=("", "_same1"))


# Merging ik_pair
final_df = final_df.merge(ik_pair[['i_uwi','k_uwi','vertical_dist','3D_ft_to_same']].rename(columns={
    'k_uwi':'k_uwi_same2',
    'vertical_dist':'vertical_dist_same2',
    '3D_ft_to_same':'3D_ft_to_same2'
}), 
               how='left', left_on=['i_uwi','k_uwi_same2'], right_on=['i_uwi','k_uwi_same2'], suffixes=("", "_same2"))


# Merging ik_pair
final_df = final_df.merge(ik_pair[['i_uwi','k_uwi','horizontal_dist','3D_ft_to_same']].rename(columns={
    'k_uwi':'k_uwi_near1',
    'horizontal_dist':'horizontal_dist_near1',
    '3D_ft_to_same':'3D_ft_to_near1'
}), 
               how='left', left_on=['i_uwi','k_uwi_near1'], right_on=['i_uwi','k_uwi_near1'], suffixes=("", "_near1"))


# Merging ik_pair
final_df = final_df.merge(ik_pair[['i_uwi','k_uwi','horizontal_dist','3D_ft_to_same']].rename(columns={
    'k_uwi':'k_uwi_near2',
    'horizontal_dist':'horizontal_dist_near2',
    '3D_ft_to_same':'3D_ft_to_near2'
}), 
               how='left', left_on=['i_uwi','k_uwi_near2'], right_on=['i_uwi','k_uwi_near2'], suffixes=("", "_near2"))


final_df = reorder_columns(final_df,['WellName', 'DSU', 'RES_CAT', 'Landing_Zone', 'FirstProdDate',
                          'k_uwi_same1','WellName_same1','horizontal_dist_same1','vertical_dist_same1','3D_ft_to_same1','RES_CAT_same1', 'Landing_Zone_same1', 'FirstProdDate_same1',
                         'k_uwi_same2','WellName_same2','horizontal_dist_same2','vertical_dist_same2','3D_ft_to_same2', 'RES_CAT_same2', 'Landing_Zone_same2', 'FirstProdDate_same2',
                         'k_uwi_near1','WellName_near1','horizontal_dist_near1','vertical_dist_near1','3D_ft_to_near1', 'RES_CAT_near1', 'Landing_Zone_near1', 'FirstProdDate_near1',
                         'k_uwi_near2','WellName_near2','horizontal_dist_near2','vertical_dist_near2','3D_ft_to_near2', 'RES_CAT_near2', 'Landing_Zone_near2', 'FirstProdDate_near2'],
                         reference_column='i_uwi')

In [26]:
final_df

,i_uwi,WellName,DSU,RES_CAT,Landing_Zone,FirstProdDate,k_uwi_same1,WellName_same1,horizontal_dist_same1,vertical_dist_same1,3D_ft_to_same1,RES_CAT_same1,Landing_Zone_same1,FirstProdDate_same1,k_uwi_same2,WellName_same2,horizontal_dist_same2,vertical_dist_same2,3D_ft_to_same2,RES_CAT_same2,Landing_Zone_same2,FirstProdDate_same2,k_uwi_near1,WellName_near1,horizontal_dist_near1,vertical_dist_near1,3D_ft_to_near1,RES_CAT_near1,Landing_Zone_near1,FirstProdDate_near1,k_uwi_near2,WellName_near2,horizontal_dist_near2,vertical_dist_near2,3D_ft_to_near2,RES_CAT_near2,Landing_Zone_near2,FirstProdDate_near2
0,4200347476,UNDERWOOD C 1H,UNDERWOOD C,01PDP,SAN ANDRES,2017-05-01,4200347639,UNDERWOOD -C- 3H,881.586121,16.328604,881.737326,01PDP,SAN ANDRES,2017-11-01,4200347590,UNDERWOOD C 2H,976.576922,6.264740,976.597016,01PDP,SAN ANDRES,2017-09-01,4222740882,MITCHELL 47-31 B UNIT L 6H,1.419920e+06,1005.023612,1.419920e+06,01PDP,LSS,2021-08-01,4203332636,SHAFER 32-12 B UNIT L 5H,1.420324e+06,1053.199013,1.420325e+06,01PDP,LSS,2023-08-01
1,4200347479,UNDERWOOD D 1H,UNDERWOOD D,01PDP,SAN ANDRES,2018-06-01,4200347647,UNDERWOOD D 3H,807.636337,1.229364,807.637273,01PDP,SAN ANDRES,2018-07-01,4200347646,UNDERWOOD D 2H,900.630835,1.180140,900.631608,01PDP,SAN ANDRES,2018-09-01,4222740882,MITCHELL 47-31 B UNIT L 6H,1.422578e+06,1019.467201,1.422578e+06,01PDP,LSS,2021-08-01,4203332636,SHAFER 32-12 B UNIT L 5H,1.422967e+06,1067.642601,1.422968e+06,01PDP,LSS,2023-08-01
2,4200347484,UNDERWOOD -B- 1H,UNDERWOOD B,01PDP,SAN ANDRES,2017-09-01,4200347640,UNDERWOOD -B- 3H,865.807922,0.410118,865.808019,01PDP,SAN ANDRES,2017-11-01,4200347609,UNDERWOOD -B- 2H,915.232283,7.197772,915.260586,01PDP,SAN ANDRES,2017-09-01,4222740882,MITCHELL 47-31 B UNIT L 6H,1.417574e+06,982.083413,1.417574e+06,01PDP,LSS,2021-08-01,4203332636,SHAFER 32-12 B UNIT L 5H,1.418000e+06,1030.258814,1.418000e+06,01PDP,LSS,2023-08-01
3,4200347590,UNDERWOOD C 2H,UNDERWOOD C,01PDP,SAN ANDRES,2017-09-01,4200347646,UNDERWOOD D 2H,890.20523,6.998708,890.232741,01PDP,SAN ANDRES,2018-09-01,4200347476,UNDERWOOD C 1H,976.576922,6.264740,976.597016,01PDP,SAN ANDRES,2017-05-01,4222740882,MITCHELL 47-31 B UNIT L 6H,1.420882e+06,1011.288352,1.420883e+06,01PDP,LSS,2021-08-01,4203332636,SHAFER 32-12 B UNIT L 5H,1.421283e+06,1059.463753,1.421284e+06,01PDP,LSS,2023-08-01
4,4200347609,UNDERWOOD -B- 2H,UNDERWOOD B,01PDP,SAN ANDRES,2017-09-01,4200347639,UNDERWOOD -C- 3H,881.458045,0.586177,881.458240,01PDP,SAN ANDRES,2017-11-01,4200347484,UNDERWOOD -B- 1H,915.232283,7.197772,915.260586,01PDP,SAN ANDRES,2017-09-01,4222740882,MITCHELL 47-31 B UNIT L 6H,1.418426e+06,989.281185,1.418427e+06,01PDP,LSS,2021-08-01,4203332636,SHAFER 32-12 B UNIT L 5H,1.418846e+06,1037.456586,1.418846e+06,01PDP,LSS,2023-08-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7629,4250137357,HOSS 605 E 2H,HOSS,01PDP,SAN ANDRES,2022-04-01,4250137527,HOSS 605 5H,2182.768283,18.767687,2182.848965,01PDP,SAN ANDRES,2024-09-01,4250137525,HOSS 605 6H,2828.693442,19.277308,2828.759128,01PDP,SAN ANDRES,2024-08-01,4222740882,MITCHELL 47-31 B UNIT L 6H,1.361407e+06,468.859857,1.361407e+06,01PDP,LSS,2021-08-01,4203332636,SHAFER 32-12 B UNIT L 5H,1.357213e+06,517.035258,1.357213e+06,01PDP,LSS,2023-08-01
7630,4250137525,HOSS 605 6H,HOSS,01PDP,SAN ANDRES,2024-08-01,4250137527,HOSS 605 5H,658.114031,0.509620,658.114228,01PDP,SAN ANDRES,2024-09-01,4250137526,HOSS 605 7H,658.73219,0.197377,658.732219,01PDP,SAN ANDRES,2024-09-01,4222740882,MITCHELL 47-31 B UNIT L 6H,1.358578e+06,449.582549,1.358578e+06,01PDP,LSS,2021-08-01,4203332636,SHAFER 32-12 B UNIT L 5H,1.354385e+06,497.75795,1.354385e+06,01PDP,LSS,2023-08-01
7631,4250137526,HOSS 605 7H,HOSS,01PDP,SAN ANDRES,2024-09-01,4250137525,HOSS 605 6H,658.73219,0.197377,658.732219,01PDP,SAN ANDRES,2024-08-01,4250137198,HOSS 605 8H,669.399283,7.806956,669.444807,01PDP,SAN ANDRES,2021-05-01,4222740882,MITCHELL

In [ ]:
# df = pd.read_csv(r"C:\Users\Apoorva.Saxena\OneDrive - Sitio Royalties\Desktop\Project - Apoorva\MB Investigation\MB_ikPairs_v1.csv")

In [ ]:
# df[df['horizontal_dist'] <= 20_000].to_csv(r"C:\Users\Apoorva.Saxena\OneDrive - Sitio Royalties\Desktop\Project - Apoorva\MB Investigation\MB_ikPairs_v3.csv", index=False)